# Setup

### Import libs.

In [ ]:
import os

import pyessv

### Set constants.

In [ ]:
_VOCAB_DOMAIN = u"earth-system.modelling"
_VOCAB_SUBDOMAIN = u"ipsl.prodiguer"    
_VOCAB_IO_DIR = os.getenv("PRODIGUER_PYESSV_HOME")
_TERMSET_COMPUTE_NODE = u"compute_node"

# Partition creation

In [ ]:
# Create vocab partition.
partition = pyessv.create_partition(
    _VOCAB_DOMAIN,
    _VOCAB_SUBDOMAIN,
    _VOCAB_IO_DIR
)

In [ ]:
# Show partitions.
pyessv.show_partitions()

# Partition cache

### Load

In [ ]:
# Loads terms from file system into memory.
partition.load()

### Counts

In [ ]:
# Cache count - all terms in all termsets;
assert partition.count() == 428

# Cache count - terms within a single termset;
assert partition.count(_TERMSET_COMPUTE_NODE) == 7

# Cache count - single term;
assert partition.count(_TERMSET_COMPUTE_NODE, "tgcc") == 1

### Access

In [ ]:
# Cache access - all termsets;
termsets = partition.query()
assert isinstance(termsets, dict)

# Cache access - single termset;
termset = partition.query(_TERMSET_COMPUTE_NODE)
assert isinstance(termset, list)

# Cache access - single term.
term = partition.query(_TERMSET_COMPUTE_NODE, "tgcc")
assert isinstance(term, pyessv.Term)

### Listing

In [ ]:
# Cache listing - all termsets;
partition.show()

In [ ]:
# Cache listing - single termset;
partition.show(_TERMSET_COMPUTE_NODE)

In [ ]:
# Cache listing - single term;
partition.show(_TERMSET_COMPUTE_NODE, "tgcc")

# Term management

### Creation

In [ ]:
# Use partition to create new terms.
term = partition.create(_TERMSET_COMPUTE_NODE, u"xxx")
assert isinstance(term, pyessv.Term)

### Validation

In [ ]:
# Invalid term - kind has invalid characters.
invalid_term = partition.create(u"&!@", u"xxx")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

In [ ]:
# Invalid term - name has invalid characters.
invalid_term = partition.create(_TERMSET_COMPUTE_NODE, u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

In [ ]:
# Invalid term - kind & name have invalid characters.
invalid_term = partition.create(u"&!@", u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 2

invalid_term.show_errors()

### Encoding

In [ ]:
# Encode as python dictionary.
as_dict = term.as_dict()
assert isinstance(as_dict(), dict)

# Encode as JSON.
as_json = term.as_json()
assert isinstance(as_json, str)

### Persistence

In [ ]:
# Saving a term writes to cache & file system.
assert term.is_cached == False
assert term.is_written == False

term.save()

assert term.is_cached == True
assert term.is_written == True

# Term Governance

In [ ]:
# What is initial governance status ?
term.status

In [ ]:
# Accept.
term.accept()

In [ ]:
# Reject.
term.reject()

In [ ]:
# Deprecate.
term.deprecate()

In [ ]:
# Destroy.
assert partition.count(_TERMSET_COMPUTE_NODE) == 8
assert partition.query(_TERMSET_COMPUTE_NODE, "xxx") is not None

term.destroy()

assert partition.count(_TERMSET_COMPUTE_NODE) == 7
assert partition.query(_TERMSET_COMPUTE_NODE, "xxx") is None

# Term Parsing

In [ ]:
# Parse a name that matches. 
name = "tgcc"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name = parsed_name

In [ ]:
# Parse a name that almost matches. 
name = "tGcC"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name != parsed_name
assert parsed_name == "tgcc"

In [ ]:
# Parse an alternative name.
alt_name = "saclay"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alt_name)
assert alt_name != parsed_name
assert parsed_name == "tgcc"

In [ ]:
# Parse an alias.
alias = "TGCC-1"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alias)
assert alias != parsed_name
assert parsed_name == "tgcc"

# Term Relationships